In [ ]:
import json
from pathlib import Path

def reshape_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    reshaped_data = []
    
    for result in data.get('results', []):
        hcfa = result.get('source_data', {}).get('hcfa', {})
        database = result.get('source_data', {}).get('database', {})
        validations = result.get('validations', {})
        rate_details = validations.get('rate', {}).get('details', {}).get('results', [])
        
        # Line items
        line_items = hcfa.get('line_items', [])
        reshaped_items = []
        for line_item in line_items:
            rate = next((r.get('rate') for r in rate_details if r.get('cpt') == line_item.get('cpt')), 0)
            reshaped_items.append({
                "cpt": line_item.get('cpt'),
                "charge": line_item.get('cleaned_charge'),
                "units": line_item.get('units', "1"),
                "modifier": line_item.get('modifier', ""),
                "date_of_service": line_item.get('cleaned_date_of_service'),
                "plos": line_item.get('plos', "11"),
                "rate": float(rate or 0.0)
            })

        # Total paid calculation
        total_paid = sum(item['rate'] for item in reshaped_items)

        # Add reshaped record
        reshaped_data.append({
            "file_name": result.get('file_name'),
            "patient_info": {
                "name": hcfa.get('patient_name'),
                "dob": database.get('order_details', {}).get('Patient_DOB'),
                "date_of_service": hcfa.get('date_of_service'),
                "address": {
                    "street": database.get('order_details', {}).get('Patient_Address'),
                    "city": database.get('order_details', {}).get('Patient_City'),
                    "state": database.get('order_details', {}).get('Patient_State'),
                    "zip": database.get('order_details', {}).get('Patient_Zip')
                },
                "injury": {
                    "date": database.get('order_details', {}).get('Patient_Injury_Date'),
                    "description": database.get('order_details', {}).get('Patient_Injury_Description')
                },
                "claim_number": database.get('order_details', {}).get('Claim_Number'),
                "FileMaker_Record_Number": database.get('order_details', {}).get('FileMaker_Record_Number'),
            },
            "provider_info": {
                "name": database.get('provider_details', {}).get('Billing Name'),
                "billing_address": {
                    "line1": database.get('provider_details', {}).get('Billing Address 1'),
                    "line2": database.get('provider_details', {}).get('Billing Address 2'),
                    "city": database.get('provider_details', {}).get('Billing Address City'),
                    "state": database.get('provider_details', {}).get('Billing Address State'),
                    "zip": database.get('provider_details', {}).get('Billing Address Postal Code'),
                    "primary_id": database.get('provider_details', {}).get('PrimaryKey')
                },
                "TIN": database.get('provider_details', {}).get('TIN'),
                "NPI": database.get('provider_details', {}).get('NPI'),
                "patient_acct_no": hcfa.get('patient_acct_no')
            },
            "line_items": reshaped_items,
            "totals": {
                "total_charge": hcfa.get('total_charge', 0),
                "total_paid": total_paid
            },
            "validations": {
                "line_items": validations.get('line_items', {}).get('messages', []),
                "rate": validations.get('rate', {}).get('messages', [])
            }
        })
    
    return reshaped_data


# Example usage
file_path = r"C:\Users\chris\OneDrive - clarity-dx.com\Documents\Bill_Review_INTERNAL\validation logs\validation_20250216_223925.json"
reshaped = reshape_json(file_path)

# Save reshaped JSON
output_path = Path(file_path).with_name("reshaped_output_with_details_20250216_01.json")
with open(output_path, 'w') as f:
    json.dump(reshaped, f, indent=2)
